In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import tensorflow as tf 
from tensorflow.keras.layers import Input, Dense, Flatten,Conv1D,MaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from sklearn.metrics import recall_score, f1_score, confusion_matrix,precision_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier


In [2]:
data = pd.read_csv("heart_disease_health_indicators_BRFSS2015.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   HeartDiseaseorAttack  253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   Diabetes              253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

In [4]:
#specify features and target columns
target = data['HeartDiseaseorAttack']
#features = data.drop(to_drop, axis=1)
features = data.drop(['HeartDiseaseorAttack'],axis=1)

In [5]:
features.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


# Adding Features Engineering 

In [6]:
features['BMI_Age'] = features['BMI'] * features['Age']
features['HighBP_Cholesterol'] = ((features['HighBP'] == 1) & (features['CholCheck'] == 1)).astype(int)

median_income = features['Income'].median()
features['HealthcareAccess'] = ((features['Income'] > median_income) & (features['AnyHealthcare'] == 1)).astype(int)



In [7]:
features.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,BMI_Age,HighBP_Cholesterol,HealthcareAccess
0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,...,18.0,15.0,1.0,0.0,9.0,4.0,3.0,360.0,1,0
1,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,6.0,1.0,175.0,0,0
2,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,...,30.0,30.0,1.0,0.0,9.0,4.0,8.0,252.0,1,1
3,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,11.0,3.0,6.0,297.0,1,0
4,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,...,3.0,0.0,0.0,0.0,11.0,5.0,4.0,264.0,1,0


# UnderSampling

In [8]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(features, target)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled ,stratify=y_resampled, test_size=0.2)

# Model 1 : DecisionTree 

In [10]:
#modelling
model=DecisionTreeClassifier(max_depth=int(5),
                             random_state=11)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
pred = precision_score(y_test, y_pred)

print("DecisionTree Classifier Accuracy: ",accuracy_score(y_test, y_pred))
print("DecisionTreeClassifier Classifier Recall: ", recall)
print("DecisionTreeClassifier F1 Score: ", f1)
print("DecisionTreeClassifier Precision: ", pred)
print("Confusion Matrix:\n", cm)


print(classification_report(y_test, y_pred))

DecisionTree Classifier Accuracy:  0.7553881565181
DecisionTreeClassifier Classifier Recall:  0.8095835948943294
DecisionTreeClassifier F1 Score:  0.7679634775704646
DecisionTreeClassifier Precision:  0.7304134415707004
Confusion Matrix:
 [[3351 1428]
 [ 910 3869]]
              precision    recall  f1-score   support

         0.0       0.79      0.70      0.74      4779
         1.0       0.73      0.81      0.77      4779

    accuracy                           0.76      9558
   macro avg       0.76      0.76      0.75      9558
weighted avg       0.76      0.76      0.75      9558



# Model 2 : Naive Bayes

In [11]:
#Basic Naive Bayes
from sklearn.naive_bayes import GaussianNB

Gnb = GaussianNB()
Gnb.fit(X_train, y_train)
y_pred = Gnb.predict(X_test)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print("Naive Bayes Classifier Accuracy: ", acc)
print("Naive Bayes Classifier Recall: ", recall)
print("Naive Bayes Classifier F1 Score: ", f1)
print("Confusion Matrix:\n", cm)


Naive Bayes Classifier Accuracy:  0.7385436283741369
Naive Bayes Classifier Recall:  0.7457627118644068
Naive Bayes Classifier F1 Score:  0.740417575568713
Confusion Matrix:
 [[3495 1284]
 [1215 3564]]


In [12]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.74      0.73      0.74      4779
         1.0       0.74      0.75      0.74      4779

    accuracy                           0.74      9558
   macro avg       0.74      0.74      0.74      9558
weighted avg       0.74      0.74      0.74      9558



# Model 3 : GradientBoostingClassifier

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import recall_score

# create GradientBoostingClassifier object
gbc = GradientBoostingClassifier()

# fit the model on training data
gbc.fit(X_train, y_train)

# predict on test data
y_pred = gbc.predict(X_test)

# calculate recall
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
pred = precision_score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print("GradientBoostingClassifier Accuracy: ", acc)
print("GradientBoostingClassifier Precision: ", pred)
print("GradientBoostingClassifier Recall: ", recall)
print("GradientBoostingClassifier F1 Score: ", f1)
print("Confusion Matrix:\n", cm)

GradientBoostingClassifier Accuracy:  0.7778824021761875
GradientBoostingClassifier Precision:  0.7542113323124043
GradientBoostingClassifier Recall:  0.8244402594685081
GradientBoostingClassifier F1 Score:  0.7877636708987305
Confusion Matrix:
 [[3495 1284]
 [ 839 3940]]


In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.81      0.73      0.77      4779
         1.0       0.75      0.82      0.79      4779

    accuracy                           0.78      9558
   macro avg       0.78      0.78      0.78      9558
weighted avg       0.78      0.78      0.78      9558



# Model 4 : Random Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score

# create RandomForestClassifier object
rfc = RandomForestClassifier()

# fit the model on training data
rfc.fit(X_train, y_train)

# predict on test data
y_pred = rfc.predict(X_test)

recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
pred = precision_score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print("RandomForestClassifier Accuracy: ", acc)
print("RandomForestClassifier Precision: ", pred)
print("RandomForestClassifier Recall: ", recall)
print("RandomForestClassifier F1 Score: ", f1)
print("Confusion Matrix:\n", cm)

RandomForestClassifier Accuracy:  0.7640719815861059
RandomForestClassifier Precision:  0.7430662557781201
RandomForestClassifier Recall:  0.8072818581293157
RandomForestClassifier F1 Score:  0.7738441480292848
Confusion Matrix:
 [[3445 1334]
 [ 921 3858]]


In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.79      0.72      0.75      4779
         1.0       0.74      0.81      0.77      4779

    accuracy                           0.76      9558
   macro avg       0.77      0.76      0.76      9558
weighted avg       0.77      0.76      0.76      9558



# Model 5 : LGBMClassifier

In [17]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, recall_score

# create LGBMClassifier object
lgbm = lgb.LGBMClassifier()

# fit the model on training data
lgbm.fit(X_train, y_train)

# predict on test data
y_pred = lgbm.predict(X_test)


recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
pred = precision_score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print("lightgbm Accuracy: ", acc)
print("lightgbm Precision: ", pred)
print("lightgbm Recall: ", recall)
print("lightgbm F1 Score: ", f1)
print("Confusion Matrix:\n", cm)


C:\Users\MSI GF 63\miniconda3\envs\tf\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


lightgbm Accuracy:  0.775162167817535
lightgbm Precision:  0.750667174990469
lightgbm Recall:  0.8240217618748692
lightgbm F1 Score:  0.7856359102244388
Confusion Matrix:
 [[3471 1308]
 [ 841 3938]]


In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.80      0.73      0.76      4779
         1.0       0.75      0.82      0.79      4779

    accuracy                           0.78      9558
   macro avg       0.78      0.78      0.77      9558
weighted avg       0.78      0.78      0.77      9558



# Model 6 : CatBoostClassifier

In [19]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

# create CatBoost classifier
catboost = CatBoostClassifier()

# train the classifier
catboost.fit(X_train, y_train)

# make predictions on the test data
y_pred = catboost.predict(X_test)

recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
pred = precision_score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print("CatBoostClassifier Accuracy: ", acc)
print("CatBoostClassifier Precision: ", pred)
print("CatBoostClassifier Recall: ", recall)
print("CatBoostClassifier F1 Score: ", f1)
print("Confusion Matrix:\n", cm)

Learning rate set to 0.048821


C:\Users\MSI GF 63\miniconda3\envs\tf\lib\site-packages\catboost\core.py:1135: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, pairs, weight, group_id, group_weight, subgroup_id, pairs_weight, baseline, feature_names, thread_count)


0:	learn: 0.6729596	total: 153ms	remaining: 2m 33s
1:	learn: 0.6544905	total: 163ms	remaining: 1m 21s
2:	learn: 0.6389111	total: 177ms	remaining: 58.7s
3:	learn: 0.6249922	total: 189ms	remaining: 47s
4:	learn: 0.6122022	total: 206ms	remaining: 41s
5:	learn: 0.6007623	total: 221ms	remaining: 36.6s
6:	learn: 0.5904092	total: 236ms	remaining: 33.5s
7:	learn: 0.5803964	total: 247ms	remaining: 30.6s
8:	learn: 0.5717703	total: 257ms	remaining: 28.3s
9:	learn: 0.5638071	total: 283ms	remaining: 28s
10:	learn: 0.5568413	total: 293ms	remaining: 26.3s
11:	learn: 0.5507364	total: 300ms	remaining: 24.7s
12:	learn: 0.5454661	total: 315ms	remaining: 23.9s
13:	learn: 0.5397845	total: 329ms	remaining: 23.2s
14:	learn: 0.5346869	total: 346ms	remaining: 22.7s
15:	learn: 0.5300520	total: 377ms	remaining: 23.2s
16:	learn: 0.5259009	total: 408ms	remaining: 23.6s
17:	learn: 0.5220525	total: 423ms	remaining: 23.1s
18:	learn: 0.5184479	total: 500ms	remaining: 25.8s
19:	learn: 0.5153631	total: 515ms	remaining: 

162:	learn: 0.4633954	total: 3.44s	remaining: 17.7s
163:	learn: 0.4632964	total: 3.45s	remaining: 17.6s
164:	learn: 0.4632291	total: 3.46s	remaining: 17.5s
165:	learn: 0.4631107	total: 3.47s	remaining: 17.4s
166:	learn: 0.4630384	total: 3.48s	remaining: 17.4s
167:	learn: 0.4629499	total: 3.5s	remaining: 17.3s
168:	learn: 0.4628723	total: 3.52s	remaining: 17.3s
169:	learn: 0.4627980	total: 3.52s	remaining: 17.2s
170:	learn: 0.4627187	total: 3.56s	remaining: 17.3s
171:	learn: 0.4626391	total: 3.58s	remaining: 17.2s
172:	learn: 0.4625504	total: 3.59s	remaining: 17.2s
173:	learn: 0.4624839	total: 3.61s	remaining: 17.1s
174:	learn: 0.4624079	total: 3.63s	remaining: 17.1s
175:	learn: 0.4623055	total: 3.64s	remaining: 17s
176:	learn: 0.4622358	total: 3.66s	remaining: 17s
177:	learn: 0.4621572	total: 3.67s	remaining: 17s
178:	learn: 0.4620852	total: 3.69s	remaining: 16.9s
179:	learn: 0.4620262	total: 3.7s	remaining: 16.9s
180:	learn: 0.4619467	total: 3.72s	remaining: 16.8s
181:	learn: 0.461881

331:	learn: 0.4510900	total: 7.2s	remaining: 14.5s
332:	learn: 0.4510626	total: 7.2s	remaining: 14.4s
333:	learn: 0.4510366	total: 7.24s	remaining: 14.4s
334:	learn: 0.4509912	total: 7.26s	remaining: 14.4s
335:	learn: 0.4509664	total: 7.27s	remaining: 14.4s
336:	learn: 0.4509385	total: 7.29s	remaining: 14.3s
337:	learn: 0.4508577	total: 7.3s	remaining: 14.3s
338:	learn: 0.4508024	total: 7.32s	remaining: 14.3s
339:	learn: 0.4507497	total: 7.33s	remaining: 14.2s
340:	learn: 0.4507053	total: 7.35s	remaining: 14.2s
341:	learn: 0.4506746	total: 7.37s	remaining: 14.2s
342:	learn: 0.4506515	total: 7.38s	remaining: 14.1s
343:	learn: 0.4505976	total: 7.39s	remaining: 14.1s
344:	learn: 0.4505442	total: 7.41s	remaining: 14.1s
345:	learn: 0.4504402	total: 7.42s	remaining: 14s
346:	learn: 0.4503678	total: 7.45s	remaining: 14s
347:	learn: 0.4503080	total: 7.46s	remaining: 14s
348:	learn: 0.4502590	total: 7.47s	remaining: 13.9s
349:	learn: 0.4501694	total: 7.49s	remaining: 13.9s
350:	learn: 0.4501413

495:	learn: 0.4419773	total: 10.5s	remaining: 10.7s
496:	learn: 0.4418981	total: 10.5s	remaining: 10.7s
497:	learn: 0.4418371	total: 10.6s	remaining: 10.6s
498:	learn: 0.4417446	total: 10.6s	remaining: 10.6s
499:	learn: 0.4416794	total: 10.6s	remaining: 10.6s
500:	learn: 0.4416412	total: 10.6s	remaining: 10.6s
501:	learn: 0.4416017	total: 10.6s	remaining: 10.5s
502:	learn: 0.4415049	total: 10.7s	remaining: 10.5s
503:	learn: 0.4414151	total: 10.7s	remaining: 10.5s
504:	learn: 0.4413522	total: 10.7s	remaining: 10.5s
505:	learn: 0.4413018	total: 10.7s	remaining: 10.4s
506:	learn: 0.4412371	total: 10.7s	remaining: 10.4s
507:	learn: 0.4412018	total: 10.7s	remaining: 10.4s
508:	learn: 0.4411515	total: 10.8s	remaining: 10.4s
509:	learn: 0.4411285	total: 10.8s	remaining: 10.4s
510:	learn: 0.4410781	total: 10.8s	remaining: 10.4s
511:	learn: 0.4410046	total: 10.8s	remaining: 10.3s
512:	learn: 0.4409248	total: 10.8s	remaining: 10.3s
513:	learn: 0.4409011	total: 10.9s	remaining: 10.3s
514:	learn: 

658:	learn: 0.4338462	total: 13.4s	remaining: 6.92s
659:	learn: 0.4337546	total: 13.4s	remaining: 6.9s
660:	learn: 0.4337174	total: 13.4s	remaining: 6.87s
661:	learn: 0.4336866	total: 13.4s	remaining: 6.85s
662:	learn: 0.4336261	total: 13.4s	remaining: 6.83s
663:	learn: 0.4335689	total: 13.4s	remaining: 6.8s
664:	learn: 0.4335058	total: 13.5s	remaining: 6.78s
665:	learn: 0.4334882	total: 13.5s	remaining: 6.75s
666:	learn: 0.4334396	total: 13.5s	remaining: 6.74s
667:	learn: 0.4333682	total: 13.6s	remaining: 6.74s
668:	learn: 0.4333170	total: 13.6s	remaining: 6.72s
669:	learn: 0.4333012	total: 13.6s	remaining: 6.69s
670:	learn: 0.4332631	total: 13.6s	remaining: 6.67s
671:	learn: 0.4332164	total: 13.6s	remaining: 6.64s
672:	learn: 0.4331782	total: 13.6s	remaining: 6.62s
673:	learn: 0.4331509	total: 13.6s	remaining: 6.6s
674:	learn: 0.4331106	total: 13.6s	remaining: 6.57s
675:	learn: 0.4330651	total: 13.7s	remaining: 6.55s
676:	learn: 0.4330291	total: 13.7s	remaining: 6.53s
677:	learn: 0.4

823:	learn: 0.4267037	total: 16.2s	remaining: 3.45s
824:	learn: 0.4266352	total: 16.2s	remaining: 3.43s
825:	learn: 0.4266120	total: 16.2s	remaining: 3.41s
826:	learn: 0.4265700	total: 16.2s	remaining: 3.38s
827:	learn: 0.4265402	total: 16.2s	remaining: 3.36s
828:	learn: 0.4264752	total: 16.2s	remaining: 3.34s
829:	learn: 0.4264603	total: 16.2s	remaining: 3.32s
830:	learn: 0.4264030	total: 16.3s	remaining: 3.31s
831:	learn: 0.4263391	total: 16.3s	remaining: 3.29s
832:	learn: 0.4262915	total: 16.3s	remaining: 3.27s
833:	learn: 0.4262138	total: 16.3s	remaining: 3.25s
834:	learn: 0.4261528	total: 16.3s	remaining: 3.23s
835:	learn: 0.4260809	total: 16.4s	remaining: 3.21s
836:	learn: 0.4260349	total: 16.4s	remaining: 3.19s
837:	learn: 0.4259673	total: 16.4s	remaining: 3.17s
838:	learn: 0.4259199	total: 16.4s	remaining: 3.14s
839:	learn: 0.4258989	total: 16.4s	remaining: 3.12s
840:	learn: 0.4258543	total: 16.4s	remaining: 3.1s
841:	learn: 0.4258070	total: 16.5s	remaining: 3.09s
842:	learn: 0

985:	learn: 0.4197261	total: 19.5s	remaining: 277ms
986:	learn: 0.4196902	total: 19.5s	remaining: 257ms
987:	learn: 0.4196615	total: 19.5s	remaining: 237ms
988:	learn: 0.4195896	total: 19.5s	remaining: 217ms
989:	learn: 0.4195786	total: 19.6s	remaining: 198ms
990:	learn: 0.4195050	total: 19.6s	remaining: 178ms
991:	learn: 0.4194512	total: 19.6s	remaining: 158ms
992:	learn: 0.4194485	total: 19.6s	remaining: 138ms
993:	learn: 0.4194308	total: 19.6s	remaining: 118ms
994:	learn: 0.4194024	total: 19.6s	remaining: 98.7ms
995:	learn: 0.4193820	total: 19.7s	remaining: 79ms
996:	learn: 0.4193308	total: 19.7s	remaining: 59.2ms
997:	learn: 0.4193006	total: 19.7s	remaining: 39.5ms
998:	learn: 0.4192824	total: 19.7s	remaining: 19.7ms
999:	learn: 0.4192777	total: 19.7s	remaining: 0us
CatBoostClassifier Accuracy:  0.7711864406779662
CatBoostClassifier Precision:  0.7480857580398163
CatBoostClassifier Recall:  0.8177442979702867
CatBoostClassifier F1 Score:  0.7813655903229032
Confusion Matrix:
 [[346

In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.80      0.72      0.76      4779
         1.0       0.75      0.82      0.78      4779

    accuracy                           0.77      9558
   macro avg       0.77      0.77      0.77      9558
weighted avg       0.77      0.77      0.77      9558



# Model 7 : XGBClassifier

In [21]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(X_train, y_train)
y_pred = xg.predict(X_test)


recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
pred = precision_score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)


print("XGBClassifier Accuracy: ", acc)
print("XGBClassifier Precision: ", pred)
print("XGBClassifier Recall: ", recall)
print("XGBClassifier F1 Score: ", f1)
print("Confusion Matrix:\n", cm)

XGBClassifier Accuracy:  0.7661644695543001
XGBClassifier Precision:  0.7458446076536529
XGBClassifier Recall:  0.8074911069261351
XGBClassifier F1 Score:  0.7754445895709837
Confusion Matrix:
 [[3464 1315]
 [ 920 3859]]


In [22]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.79      0.72      0.76      4779
         1.0       0.75      0.81      0.78      4779

    accuracy                           0.77      9558
   macro avg       0.77      0.77      0.77      9558
weighted avg       0.77      0.77      0.77      9558



# CNN

In [23]:
import numpy as np
X_train_2d=np.expand_dims(X_train,axis=2)
n_timesteps,n_features=X_train_2d.shape[1],X_train_2d.shape[2]
#CNN Initialization
model=Sequential()
model.add(Conv1D(filters=16,kernel_size=2,activation="relu",input_shape=(n_timesteps,n_features)))
#CNN Pooling
model.add(MaxPooling1D(pool_size=2))
#flatten layer
model.add(Flatten())
#Fully connected Layer
model.add(Dense(16,activation="relu"))
#Decision Layer
model.add(Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 23, 16)            48        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 11, 16)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 176)               0         
                                                                 
 dense (Dense)               (None, 16)                2832      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 2,897
Trainable params: 2,897
Non-trainable params: 0
______________________________________________________

# Recall

In [25]:
from tensorflow.keras.metrics import Recall

model.compile(loss = 'binary_crossentropy',optimizer = tf.keras.optimizers.Adam(lr=0.001),
                metrics = [Recall()])

history = model.fit(X_train_2d, y_train,validation_split=0.34,shuffle=True,epochs=25,batch_size=64,verbose=1)

Epoch 1/25
395/395 [==============================] - 2s 4ms/step - loss: 0.4970 - recall_1: 0.7916 - val_loss: 0.4829 - val_recall_1: 0.7855
Epoch 2/25
395/395 [==============================] - 1s 4ms/step - loss: 0.4887 - recall_1: 0.8044 - val_loss: 0.4996 - val_recall_1: 0.6766
Epoch 3/25
395/395 [==============================] - 1s 4ms/step - loss: 0.4965 - recall_1: 0.7854 - val_loss: 0.4884 - val_recall_1: 0.7295
Epoch 4/25
395/395 [==============================] - 1s 4ms/step - loss: 0.4938 - recall_1: 0.7909 - val_loss: 0.5136 - val_recall_1: 0.8981
Epoch 5/25
395/395 [==============================] - 1s 4ms/step - loss: 0.4922 - recall_1: 0.7917 - val_loss: 0.4807 - val_recall_1: 0.7928
Epoch 6/25
395/395 [==============================] - 1s 4ms/step - loss: 0.4903 - recall_1: 0.7933 - val_loss: 0.4816 - val_recall_1: 0.7699
Epoch 7/25
395/395 [==============================] - 1s 4ms/step - loss: 0.4884 - recall_1: 0.8003 - val_loss: 0.4814 - val_recall_1: 0.8319
Epoch 

# Accuracy

In [26]:
model.compile(loss = 'binary_crossentropy',optimizer = tf.keras.optimizers.Adam(lr=0.009),
                metrics = ['accuracy'])
history = model.fit(X_train_2d, y_train,validation_split=0.34,shuffle=True,epochs=25,batch_size=64,verbose=1)

Epoch 1/25
395/395 [==============================] - 2s 5ms/step - loss: 0.5097 - accuracy: 0.7524 - val_loss: 0.4915 - val_accuracy: 0.7626
Epoch 2/25
395/395 [==============================] - 2s 5ms/step - loss: 0.4976 - accuracy: 0.7560 - val_loss: 0.4906 - val_accuracy: 0.7669
Epoch 3/25
395/395 [==============================] - 2s 5ms/step - loss: 0.4944 - accuracy: 0.7611 - val_loss: 0.4829 - val_accuracy: 0.7713
Epoch 4/25
395/395 [==============================] - 2s 5ms/step - loss: 0.4897 - accuracy: 0.7635 - val_loss: 0.5075 - val_accuracy: 0.7563
Epoch 5/25
395/395 [==============================] - 2s 4ms/step - loss: 0.4926 - accuracy: 0.7636 - val_loss: 0.4886 - val_accuracy: 0.7665
Epoch 6/25
395/395 [==============================] - 2s 5ms/step - loss: 0.4924 - accuracy: 0.7625 - val_loss: 0.4914 - val_accuracy: 0.7660
Epoch 7/25
395/395 [==============================] - 2s 4ms/step - loss: 0.4901 - accuracy: 0.7635 - val_loss: 0.4871 - val_accuracy: 0.7691
Epoch 

# Precision

In [27]:
from tensorflow.keras.metrics import Precision

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=[Precision()])

history = model.fit(X_train_2d, y_train, validation_split=0.34, shuffle=True, epochs=25, batch_size=64, verbose=1)

Epoch 1/25
395/395 [==============================] - 3s 5ms/step - loss: 0.4801 - precision: 0.7502 - val_loss: 0.4813 - val_precision: 0.7391
Epoch 2/25
395/395 [==============================] - 2s 5ms/step - loss: 0.4792 - precision: 0.7493 - val_loss: 0.4804 - val_precision: 0.7455
Epoch 3/25
395/395 [==============================] - 2s 5ms/step - loss: 0.4787 - precision: 0.7515 - val_loss: 0.4814 - val_precision: 0.7411
Epoch 4/25
395/395 [==============================] - 2s 5ms/step - loss: 0.4788 - precision: 0.7487 - val_loss: 0.4802 - val_precision: 0.7474
Epoch 5/25
395/395 [==============================] - 2s 5ms/step - loss: 0.4785 - precision: 0.7507 - val_loss: 0.4803 - val_precision: 0.7416
Epoch 6/25
395/395 [==============================] - 2s 5ms/step - loss: 0.4785 - precision: 0.7503 - val_loss: 0.4800 - val_precision: 0.7425
Epoch 7/25
395/395 [==============================] - 2s 5ms/step - loss: 0.4783 - precision: 0.7515 - val_loss: 0.4801 - val_precision: